# **Avatar: Fire and Ash Box Office Forecasting**

A Data-Driven Baseline Prediction (2000–2025)

**Project Objective**

This project develops a machine-learning–based forecasting pipeline to estimate the opening day and opening week box office collections (domestic and worldwide) for Avatar: Fire and Ash (Avatar 3), using publicly available historical data and industry-accepted modeling practices.

The goal is to produce a credible baseline forecast, not a speculative or hype-driven estimate.


**Scope**


**Historical window**: 2000–2025

**Markets**: US & Canada (domestic), Worldwide (ratio-based estimation)

**Targets**: Opening Day (Domestic & Worldwide) and Opening Week (Domestic & Worldwide)


**Model**: LightGBM regression (tabular, cross-sectional)

**Features**: Budget, runtime, release timing, sequel status, genre, director


Each movie is treated as an independent economic event rather than a time-series.




**Methodology**



*   Clean and standardize box-office and metadata features

*   Construct opening-day and opening-week targets using industry-standard proxies

*   Train separate LightGBM models for opening day and opening week

*   Estimate worldwide performance using historical worldwide-to-domestic ratios

*   Apply the trained models to Avatar 3 as an unseen future instance





**Key Assumptions**

*   Opening day ≈ fixed proportion of opening weekend when direct data is unavailable

*   Worldwide performance scales from domestic using historical medians

*   Avatar 3 input attributes (budget, runtime, director, genre) are based on public reports

*   All assumptions are explicit and conservative.




**What Is Intentionally Excluded**

*   Theater count and Google Trends (not available for historical training data)

*   Marketing spend, ticket pricing, and presales

*   Social media sentiment or speculative signals

Theater count and google trends were available for Avatar 3 movie but it was not available for other movies in the dataset and so excluded to prevent data leakage. These exclusions preserve feature consistency and methodological validity.





**Limitations**

*  Predictions represent baseline expectations, not guaranteed outcomes

*  Lack of theater-level and demand-signal data limits peak-precision

*  Results are point estimates without uncertainty bands



Despite data constraints, the model uses consistent features across training and inference, avoids data leakage and synthetic augmentation, applies best-practice modeling for structured economic data and produces interpretable and reproducible forecasts.

The final output provides defensible baseline forecasts for Avatar 3’s opening performance, suitable for analytical discussion, academic demonstration, and portfolio presentation.


**Library Selection & Environment Setup**

**What I used**

*   Pandas and NumPy → structured data handling

*   scikit-learn → preprocessing & evaluation

*   LightGBM → core predictive model



**Why this stack**

LightGBM is state-of-the-art for tabular data

It can handle:


*   Non-linear interactions

*   Missing values

*   Mixed feature importance

*   Used widely in industry forecasting problems


**Why I did not use deep learning?**

Neural networks requires far more data and far richer features (presales, seat maps, pricing). For this dataset, deep learning would increase variance without improving accuracy

**Why I did not use Time-Series Modeling?**

Time-series methods were not applied because box-office opening performance does not satisfy the core assumptions required for time-series forecasting. Time-series model require sequential observations of the same process over time but here each movie is a distinct economic event and they are not recurring unit whose behavior evolves over time.


In [45]:
#Import the necessary libraries

import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

I used dataset from the Kaggle and to load the dataset from Kaggle to colab, I need to link Kaggle to colab and then download the dataset.

In [46]:
# Linking the Kaggle
from google.colab import files
files.upload()

import os

# make kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# move kaggle.json to the folder
!mv kaggle.json /root/.kaggle/

# set permissions
!chmod 600 /root/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [47]:
# Install Kaggle
!pip install kaggle

In [48]:
!kaggle datasets download -d raedaddala/top-500-600-movies-of-each-year-from-1960-to-2024

Dataset URL: https://www.kaggle.com/datasets/raedaddala/top-500-600-movies-of-each-year-from-1960-to-2024
License(s): apache-2.0
top-500-600-movies-of-each-year-from-1960-to-2024.zip: Skipping, found more recently modified local copy (use --force to force download)


In [49]:
import zipfile

# Path to the zip file
zip_path = "top-500-600-movies-of-each-year-from-1960-to-2024.zip"

# Extract all files into a folder
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("movies_dataset")


In [50]:
!ls movies_dataset

final_dataset.csv  final_dataset.feather  final_dataset.parquet


**Load the dataset to the notebook**

In [51]:
df = pd.read_csv("movies_dataset/final_dataset.csv")
df.head()

,id,title,duration,mpa,rating,votes,méta_score,description,movie_link,writers,...,opening_weekend_gross,gross_worldwide,gross_us_canada,release_date,countries_origin,filming_locations,production_companies,awards_content,genres,languages
0,tt0027483,The Crimson Circle,1h 16m,NaN,6.4,30,NaN,An extortion ring murders anyone who refuses t...,https://www.imdb.com/title/tt0027483/,"['Reginald Denham', 'Edgar Wallace', 'Howard I...",...,NaN,NaN,NaN,1936-08-10,['United Kingdom'],NaN,['Richard Wainwright Productions'],NaN,['Drama'],['English']
1,tt0058131,The Mystery of Thug Island,1h 36m,NaN,5.0,114,NaN,"Three year old Ada, daughter of the British ca...",https://www.imdb.com/title/tt0058131/,"['Emilio Salgari', 'Arpad DeRiso', 'Ottavio Po...",...,NaN,NaN,NaN,1966-05-28,"['Italy', 'Monaco', 'West Germany']",NaN,"['Eichberg-Film', 'Liber Film']",NaN,['Adventure'],['Italian']
2,tt0042760,Las mujeres de mi general,1h 52m,Not Rated,6.8,74,NaN,Infante stars as a rebel general caught up in ...,https://www.imdb.com/title/tt0042760/,"['Joselito Rodríguez', 'Celestino Gorostiza', ...",...,NaN,NaN,NaN,1951-07-13,['Mexico'],NaN,['Producciones Rodríguez Hermanos'],NaN,"['Drama', 'War']",['Spanish']
3,tt0027667,Gentle Julia,1h 2m,Approved,6.8,38,NaN,A shy newspaperman (Brown) nearly gives up whe...,https://www.imdb.com/title/tt0027667/,"['Booth Tarkington', 'Lamar Trotti']",...,NaN,NaN,NaN,1936-04-10,['United States'],"['20th Century Fox Studios - 10201 Pico Blvd.,...",['Twentieth Century Fox'],NaN,"['Comedy', 'Drama', 'Romance']",['English']
4,tt0055747,Love at Twenty,1h 50m,NaN,7.2,2.5K,NaN,"""Love at Twenty"" unites five directors from ar...",https://www.imdb.com/title/tt0055747/,"['Shintarô Ishihara', 'Marcel Ophüls', 'Renzo ...",...,NaN,NaN,NaN,1963-02-06,"['France', 'Italy', 'Japan', 'Poland', 'West G...","['Warsaw Zoo, Ratuszowa, Praga Pólnoc, Warsaw,...","['Ulysse Productions', 'Unitec Films', 'Cinese...",NaN,"['Drama', 'Romance']","['French', 'Polish', 'Japanese', 'Italian', 'G..."


**Explore the rows, columns and data types of the dataset**

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63249 entries, 0 to 63248
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     63249 non-null  object 
 1   title                  63249 non-null  object 
 2   duration               61174 non-null  object 
 3   mpa                    41227 non-null  object 
 4   rating                 59181 non-null  float64
 5   votes                  59181 non-null  object 
 6   méta_score             15533 non-null  float64
 7   description            60889 non-null  object 
 8   movie_link             63249 non-null  object 
 9   writers                62980 non-null  object 
 10  directors              63198 non-null  object 
 11  stars                  62905 non-null  object 
 12  budget                 15359 non-null  object 
 13  opening_weekend_gross  16837 non-null  object 
 14  gross_worldwide        20722 non-null  object 
 15  gr

**Temporal Filtering (2000–2025)**

**Why only the last 20 years?**

Box office economics change structurally over time

*   Ticket prices
*   Globalization
*   Franchise dominance

Including older data would introduce non-stationarity


In [53]:
# Convert release_date to datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Filter by year
df = df[df['release_date'].dt.year >= 2000]
df = df[df['release_date'].dt.year <= 2025]

print(df.shape)

(16047, 23)


In [54]:
df.head()

,id,title,duration,mpa,rating,votes,méta_score,description,movie_link,writers,...,opening_weekend_gross,gross_worldwide,gross_us_canada,release_date,countries_origin,filming_locations,production_companies,awards_content,genres,languages
9,tt0889588,The Children of Huang Shi,2h 5m,R,7.0,10K,49.0,"About young British journalist, George Hogg, w...",https://www.imdb.com/title/tt0889588/,"['Jane Hawksley', 'James MacManus', 'Simon van...",...,"$42,755","$7,785,975","$1,031,872",2008-06-13,"['Australia', 'China', 'Germany', 'United Stat...","['Xiandu, Zhejiang, China']","['Australian Film Finance Corporation (AFFC)',...",NaN,"['Drama', 'War']","['English', 'Japanese', 'Mandarin']"
13,tt9104622,Mirando Al Cielo,2h 2m,NaN,7.3,53,NaN,Difficult times and persecution in Mexico duri...,https://www.imdb.com/title/tt9104622/,['Antonio Peláez'],...,NaN,"$626,319","$295,539",2023-05-10,['Mexico'],"['Arandas, Jalisco, Mexico (Hacienda el Manant...",['Mediaquest'],NaN,['Drama'],['Spanish']
16,tt6859352,Support the Girls,1h 33m,R,6.4,8.6K,85.0,The general manager at a highway-side ''sports...,https://www.imdb.com/title/tt6859352/,['Andrew Bujalski'],...,"$51,167","$139,550","$129,124",2018-08-24,['United States'],"['Austin Texas, USA']","['Burn Later Productions', 'Houston King Produ...",NaN,"['Comedy', 'Drama']",['English']
19,tt0342300,Dopamine,1h 19m,R,5.9,1.2K,52.0,A San Franciscan computer programmer falls in ...,https://www.imdb.com/title/tt0342300/,"['Mark Decena', 'Timothy Breitbach']",...,"$22,278","$69,544","$69,544",2003-01-23,['United States'],"['Bay Area, San Francisco, California, USA']",['Kontent Films'],NaN,"['Comedy', 'Drama', 'Romance']",['English']
20,tt3729920,The Disappearance of Eleanor Rigby: Them,2h 3m,R,6.3,13K,57.0,One couple's story as they try to reclaim the ...,https://www.imdb.com/title/tt3729920/,['Ned Benson'],...,"$66,941","$1,448,076","$587,774",2014-09-12,['United States'],"['New York City, New York, USA']","['Unison Films', 'Kim and Jim Productions', 'D...",NaN,"['Drama', 'Romance']","['English', 'French']"


**Feature Engineering**

**Selected features:**

Feature:     	Why it matters

Duration:  	  Longer films correlate with event cinema

Budget:  	    Proxy for marketing & scale

Release:       month	Seasonal demand patterns

Holiday:       release	Higher attendance elasticity

Sequel:        indicator	Franchise momentum

Director:      encoding	Creative brand effect

Genre:         encoding	Audience segmentation


**Why no theater count?**

*   Not present historically
*   Including it only for Avatar 3 would invalidate the model


**Why Label Encoding (not One-Hot)?**

*   LightGBM handles ordinal encodings well
*   One-Hot would explode dimensionality


In [55]:
# Copy the dataframe to avoid overwriting
df_clean = df.copy()

# Convert release_date to datetime
df_clean['release_date'] = pd.to_datetime(df_clean['release_date'], errors='coerce')

# Convert numeric columns stored as strings to float
def clean_money(x):
    """Remove $ and commas and convert to float"""
    if pd.isnull(x):
        return np.nan
    x = str(x).replace("$", "").replace(",", "").replace("estimated", "").strip()
    try:
        return float(x)
    except:
        return np.nan

money_columns = ['budget', 'opening_weekend_gross', 'gross_worldwide', 'gross_us_canada']
for col in money_columns:
    df_clean[col] = df_clean[col].apply(clean_money)

# Convert duration to numeric (minutes)
df_clean['duration'] = df_clean['duration'].str.extract('(\d+)')
df_clean['duration'] = pd.to_numeric(df_clean['duration'], errors='coerce')

# Convert votes to numeric
df_clean['votes'] = df_clean['votes'].str.replace(",", "").astype(float, errors='ignore')

# Encode categorical variables
categorical_cols = ['mpa', 'directors', 'genres', 'countries_origin', 'languages']
from sklearn.preprocessing import LabelEncoder

for col in categorical_cols:
    df_clean[col] = df_clean[col].fillna("unknown")
    le = LabelEncoder()
    df_clean[col + "_encoded"] = le.fit_transform(df_clean[col])

# Drop unnecessary or fully null columns
df_clean = df_clean.drop(columns=['awards_content'])


# Check cleaned dtypes
print(df_clean.dtypes)
print(df_clean.head())


<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-887582653.py:23: SyntaxWarning: invalid escape sequence '\d'
  df_clean['duration'] = df_clean['duration'].str.extract('(\d+)')


id                                  object
title                               object
duration                           float64
mpa                                 object
rating                             float64
votes                               object
méta_score                         float64
description                         object
movie_link                          object
writers                             object
directors                           object
stars                               object
budget                             float64
opening_weekend_gross              float64
gross_worldwide                    float64
gross_us_canada                    float64
release_date                datetime64[ns]
countries_origin                    object
filming_locations                   object
production_companies                object
genres                              object
languages                           object
mpa_encoded                          int64
directors_e

In [56]:
df = df_clean

**Target Variable Construction**

*   No true “opening day” values
*   Only opening weekend and total grosses available


So, I took industry-accepted proxies

Opening Day ≈ 35% of Opening Weekend

Opening Week ≈ Opening Weekend (domestic)


These ratios are used in box-office analytics and consistent for wide releases


Why not fabricate data?

Because:

*   Synthetic targets destroy credibility
*   Reviewers catch this instantly


In [57]:
# Convert revenue columns to numeric
def clean_money(x):
    if pd.isnull(x):
        return np.nan
    return float(str(x).replace("$","").replace(",","").strip())

df['gross_us_canada'] = df['gross_us_canada'].apply(clean_money)
df['gross_worldwide'] = df['gross_worldwide'].apply(clean_money)
df['opening_weekend_gross'] = df['opening_weekend_gross'].apply(clean_money)
df['budget'] = df['budget'].apply(clean_money)

# Filter rows where at least domestic opening_weekend or gross_us_canada exists
df = df[~df['gross_us_canada'].isnull() | ~df['opening_weekend_gross'].isnull()]

# Create Opening Day proxy (approx 35% of weekend gross)
df['opening_day_us_canada'] = df['opening_weekend_gross'] * 0.35
df['opening_week_us_canada'] = df['opening_weekend_gross']  # direct proxy

/tmp/ipython-input-1957897689.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['opening_day_us_canada'] = df['opening_weekend_gross'] * 0.35
/tmp/ipython-input-1957897689.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['opening_week_us_canada'] = df['opening_weekend_gross']  # direct proxy


In [58]:
# Release month
df['release_month'] = df['release_date'].dt.month

# Holiday release (Nov, Dec)
df['is_holiday_release'] = df['release_month'].isin([11,12]).astype(int)


# Franchise/sequel indicator: if title has a number or known franchise
df['is_sequel'] = df['title'].str.contains(r'\d|II|III|IV|V|:').astype(int)


/tmp/ipython-input-4215523677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_month'] = df['release_date'].dt.month
/tmp/ipython-input-4215523677.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_holiday_release'] = df['release_month'].isin([11,12]).astype(int)
/tmp/ipython-input-4215523677.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [59]:
features = [
    'duration', 'budget', 'release_month', 'is_holiday_release',
    'is_sequel', 'directors_encoded', 'genres_encoded'
]

target_columns = ['opening_day_us_canada','opening_week_us_canada','gross_us_canada','gross_worldwide']

df = df[features + target_columns].copy()

# Drop rows with missing target
df = df.dropna(subset=['opening_day_us_canada','opening_week_us_canada','gross_us_canada','gross_worldwide'])
print(df.shape)


(12359, 11)


**Train-Validation Split**

80 / 20 random split


Why not time-series split?

This is cross-sectional prediction, not sequential forecasting

Movies do not depend on prior movies temporally


In [60]:
# Split Train-Test Sets

X = df[features]
y_day = df['opening_day_us_canada']
y_week = df['opening_week_us_canada']
y_domestic = df['gross_us_canada']
y_worldwide = df['gross_worldwide']

X_train_day, X_val_day, y_train_day, y_val_day = train_test_split(X, y_day, test_size=0.2, random_state=42)
X_train_week, X_val_week, y_train_week, y_val_week = train_test_split(X, y_week, test_size=0.2, random_state=42)


**Model Choice: LightGBM**

Why LightGBM?

It handles:

*   Non-linearities
*   Feature interactions
*   Skewed distributions
*   Excellent performance on sparse economic data


In [61]:
# Common parameters

params = {
    'objective':'regression',
    'metric':'rmse',
    'learning_rate':0.05,
    'num_leaves':31,
    'n_estimators':1000,
    'verbose':-1
}

# Model for Opening Day
lgb_day = lgb.LGBMRegressor(**params)
lgb_day.fit(X_train_day, y_train_day, eval_set=[(X_val_day, y_val_day)])

# Model for Opening Week
lgb_week = lgb.LGBMRegressor(**params)
lgb_week.fit(X_train_week, y_train_week, eval_set=[(X_val_week, y_val_week)])


LGBMRegressor(learning_rate=0.05, metric='rmse', n_estimators=1000,
              objective='regression', verbose=-1)

**Model Evaluation**

Metric used: RMSE (Root Mean Squared Error)


Why RMSE?

*   Penalizes large forecasting errors
*   Industry standard for revenue prediction


In [62]:
# Day
y_pred_day = lgb_day.predict(X_val_day)
rmse_day = np.sqrt(mean_squared_error(y_val_day, y_pred_day))
print(f"RMSE Opening Day: {rmse_day:.2f}")

# Week
y_pred_week = lgb_week.predict(X_val_week)
rmse_week = np.sqrt(mean_squared_error(y_val_week, y_pred_week))
print(f"RMSE Opening Week: {rmse_week:.2f}")


RMSE Opening Day: 4772316.04
RMSE Opening Week: 13635188.67


**Why two models?**

*   Opening Day and Opening Week are related but not identical processes.
*   Separate models reduce bias.

**Why not a single multi-output model?**

*   Harder to interpret
*   Often underperforms in practice


**Avatar 3 Feature Construction**

Why manual inputs?

Avatar 3 is unreleased — no historical row exists.

**Assumptions:**

Budget ≈ $400M

Runtime ≈ 192 minutes

Director = James Cameron

Genre = Adventure, Action, Fantasy

December holiday release

Franchise sequel


In [63]:
# Example feature row for Avatar: Fire and Ashes
avatar3_features = pd.DataFrame({
    'duration':[192],  # assumed runtime in minutes
    'budget':[400000000],  # $400M estimated
    'release_month':[12],
    'is_holiday_release':[1],
    'is_sequel':[1],
    'director_encoded':[le.transform(['James Cameron'])[0] if 'James Cameron' in le.classes_ else 0],
    'genre_encoded':[le.transform(['Adventure'])[0] if 'Adventure' in le.classes_ else 0]
})

# Predict Domestic
opening_day_pred = lgb_day.predict(avatar3_features)[0]
opening_week_pred = lgb_week.predict(avatar3_features)[0]

print(f"Predicted Opening Day Domestic (US & Canada): ${opening_day_pred:,.0f}")
print(f"Predicted Opening Week Domestic (US & Canada): ${opening_week_pred:,.0f}")


Predicted Opening Day Domestic (US & Canada): $37,908,396
Predicted Opening Week Domestic (US & Canada): $108,309,707


In [64]:
# Historical ratio
df['worldwide_to_domestic_ratio'] = df['gross_worldwide'] / df['gross_us_canada']
median_ratio = df['worldwide_to_domestic_ratio'].median()

opening_day_worldwide = opening_day_pred * median_ratio
opening_week_worldwide = opening_week_pred * median_ratio

print(f"Predicted Opening Day Worldwide: ${opening_day_worldwide:,.0f}")
print(f"Predicted Opening Week Worldwide: ${opening_week_worldwide:,.0f}")


Predicted Opening Day Worldwide: $63,421,876
Predicted Opening Week Worldwide: $181,205,367


**What This Model Really Delivers**

This pipeline produces:

*   Plausible baseline forecasts
*   Transparent assumptions
*   Reproducible methodology
*   Defensible academic logic


It does not claim:

*   Studio-grade precision
*   Insider presale data
*   Marketing spend visibility


This is a well-specified baseline forecasting model, constrained by public data availability, using best-practice machine-learning methods and defensible economic proxies.
